正则就是在模型的损失函数当中来添加惩罚来限制模型参数的大小，从而提升模型的泛化能力。
l1正则也就是Lasso正则是通过在损失函数中加入参数的绝对值之和来作为惩罚
Loss = Original Loss + 正则化参数用于控制惩罚项的强度；然后乘以wi也就是模型的权重参数的绝对值.

L2正则也就是Ridge正则是通过在损失函数中加入参数的平方和作为惩罚项。
Loss = Original Loss + 正则化参数，用于控制惩罚项的强度，然后是权重参数的平方。

数据生成

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

# 生成一些示例数据
torch.manual_seed(0)
X = torch.randn(100, 1)
y = 3 * X.squeeze() + 2 + 0.5 * torch.randn(100)

# 定义数据集和数据加载器
dataset = torch.utils.data.TensorDataset(X, y)
data_loader = torch.utils.data.DataLoader(dataset, batch_size=10, shuffle=True)

定义模型和损失函数

In [3]:
# 定义线性回归模型
class LinearRegressionModel(nn.Module):
    def __init__(self):
        super(LinearRegressionModel, self).__init__()
        self.linear = nn.Linear(1, 1)

    def forward(self, x):
        return self.linear(x)

model = LinearRegressionModel()
criterion = nn.MSELoss()

L1正则化

In [ ]:
# 使用L1正则化
optimizer = optim.SGD(model.parameters(), lr=0.01)

# 定义L1正则化的强度
l1_lambda = 0.01

# 训练模型
num_epochs = 100
for epoch in range(num_epochs):
    for inputs, targets in data_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets.unsqueeze(1))

        # 添加L1正则化项
        l1_norm = sum(p.abs().sum() for p in model.parameters())
        loss = loss + l1_lambda * l1_norm

        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}, Loss: {loss.item()}")

L2正则化

In [ ]:
# 使用L2正则化
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=0.01)  # weight_decay用于L2正则化

# 训练模型
num_epochs = 100
for epoch in range(num_epochs):
    for inputs, targets in data_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets.unsqueeze(1))
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}, Loss: {loss.item()}")


总结
L1正则化通过添加权重的绝对值之和，促进稀疏性，可能会导致一些权重变成0，从来自动进行特征选择
L2正则化通过添加权重的平方和，限制权重的大小，是模型更加平滑，防止权重过大，减少过拟合风险

如果说不想选我都想要那就用Elastic Net来实现这就是把俩都加一起了

In [6]:
from sklearn.linear_model import ElasticNet
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split

# 生成示例数据
X, y = make_regression(n_samples=1000, n_features=20, noise=0.1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 定义Elastic Net模型
elastic_net = ElasticNet(alpha=1.0, l1_ratio=0.5)  # l1_ratio=0.5表示L1和L2正则化的混合比例

# 训练模型
elastic_net.fit(X_train, y_train)

# 评估模型
print(f"Train Score: {elastic_net.score(X_train, y_train)}")
print(f"Test Score: {elastic_net.score(X_test, y_test)}")


Train Score: 0.8900192352862634
Test Score: 0.8853356711767842


以下是一个自动选择特征的例子

In [8]:
from sklearn.linear_model import Lasso
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
import numpy as np

# 加载加利福尼亚房价数据集
data = fetch_california_housing()
X, y = data.data, data.target

# 拆分数据集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 定义Lasso回归模型
lasso = Lasso(alpha=0.1)

# 训练模型
lasso.fit(X_train, y_train)

# 输出特征权重
print("Feature weights:", lasso.coef_)

# 查看被选择的重要特征
selected_features = np.where(lasso.coef_ != 0)[0]
print("Selected features indices:", selected_features)
print("Selected features names:", [data.feature_names[i] for i in selected_features])

# 评估模型
train_score = lasso.score(X_train, y_train)
test_score = lasso.score(X_test, y_test)
print(f"Train Score: {train_score}")
print(f"Test Score: {test_score}")


Feature weights: [ 3.92693362e-01  1.50810624e-02 -0.00000000e+00  0.00000000e+00
  1.64168387e-05 -3.14918929e-03 -1.14291203e-01 -9.93076483e-02]
Selected features indices: [0 1 4 5 6 7]
Selected features names: ['MedInc', 'HouseAge', 'Population', 'AveOccup', 'Latitude', 'Longitude']
Train Score: 0.5489153425707493
Test Score: 0.5318167610318159
